In [16]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.append('/Users/ajith.a/source_code/arango-ch/UGC/Arango-CH')
sys.path.append('/Users/ajith.a/source_code/arango-ch/UGC/Arango-CH/venv/lib/python3.7/site-packages')

Python 3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) 
[Clang 6.0 (clang-600.0.57)] on darwin


In [17]:
import os
import sys
import traceback
from pathlib import Path

import click
from clickhouse_driver import Client
import pandas as pd

In [18]:
from arango_db.connect import ArangoHelper, get_arango_client_using_dict_config
from cache.connect import get_redis_client
from clickhouse.connect import ClickhouseHelper, get_ch_client_with_dict_config
from clickhouse.helper import drop_table_if_exists, rename_table
from config import env_reader, config_reader
from logger import logger

In [19]:
env_reader.load_env()
config_env = os.getenv('env')
config = config_reader.load_config(config_env)
path = Path(os.path.realpath(sys.argv[0])).parent
logging = logger.initialize_logger(config_env, path, config_env == 'dev')

In [20]:
import synchronizer.loader.populate as populate

In [21]:
from synchronizer.consumer.schema.schemas import get_table_map_by_arango_collection, get_primary_key_map

In [22]:
def get_all_documents(client, col):
    result = []
    for docs in populate.get_all_documents(client, col):
        result.extend(docs)
    return result

In [23]:
def get_arano_documents_by_query(db_client: ArangoHelper, col_name, query, batch_size=10000):
    for documents in db_client.select_using_yield(query, batch_size=batch_size, ttl=1800):
        yield documents

In [24]:
def get_documents_by_query(client, col, query):
    result = []
    for docs in get_arano_documents_by_query(client, col, query):
        result.extend(docs)
    return result

In [25]:
arango_client = get_arango_client_using_dict_config(config['arango'])

In [26]:
clickhouse: ClickhouseHelper = get_ch_client_with_dict_config(config['clickhouse'])
clickhouse_client: Client = clickhouse.client

In [86]:
# compare collection
def compare_collection(collection, query, compare):
    table_map = get_table_map_by_arango_collection(collection)
    schema = table_map['schema']
    ch_data = clickhouse_client.execute(f'''
        SELECT *
        FROM {table_map['clickhouse']}
        FORMAT JSON
    ''', with_column_types=True)
    arango_data = get_documents_by_query(arango_client, table_map['arango'], query)
    clickhouse_data = [{ch_data[1][i][0]: c for i,c in enumerate(r)} for r in ch_data[0]]
    column_map = {v['ref'] if 'ref' in v else k:k for k, v in schema.items()}
    arango_df = pd.DataFrame(arango_data)
    arango_df = arango_df.rename(columns=column_map)
    clickhouse_data_df = pd.DataFrame(clickhouse_data)
    print(list(column_map.values()))
    print(arango_df.info())
    print(clickhouse_data_df.info())
    clickhouse_data_df = clickhouse_data_df[compare]
    arango_df = arango_df[clickhouse_data_df.columns]
    p_key_map = get_primary_key_map()
    primary_key = p_key_map[table_map['clickhouse']]
    arango_df[primary_key] = arango_df[primary_key].astype(str(clickhouse_data_df[primary_key].dtype))
    arango_df = arango_df.loc[:,~arango_df.columns.duplicated()]
    common = pd.merge(arango_df, clickhouse_data_df, on=primary_key)
    left_only = pd.merge(arango_df, clickhouse_data_df, on=primary_key, how='left', indicator=True)
    left_only = left_only[left_only['_merge'] == 'left_only']
    right_only = pd.merge(arango_df, clickhouse_data_df, on=primary_key, how='right', indicator=True)
    right_only = right_only[right_only['_merge'] == 'right_only']
    mismatch = pd.merge(arango_df, clickhouse_data_df, on=primary_key, how='left', indicator=True)
    mismatch = mismatch[mismatch['_merge'] == 'both']
    mismatch = mismatch[~mismatch[primary_key].isin(common[primary_key])]
    duplicates = len(clickhouse_data_df[primary_key].unique()) < len(clickhouse_data_df)
    return {
        'arango': arango_data,
        'clickhouse': clickhouse_data,
        'arango_df': arango_df,
        'clickhouse_df': clickhouse_data_df,
        'duplicates': duplicates,
        'arango_len': len(arango_data),
        'clickhouse_len': len(clickhouse_data),
        'common': common,
        'common_len': len(common),
        'left_only': left_only,
        'left_only_len': len(left_only),
        'right_only': right_only,
        'right_only_len': len(right_only),
        'mismatch': mismatch,
        'mismatch_len': len(mismatch)
    }

In [61]:
def get_ch_columns(collection):
    table_map = get_table_map_by_arango_collection(collection)
    schema = table_map['schema']
    column_map = {v['ref'] if 'ref' in v else k: k for k, v in schema.items()}
    return column_map, list(column_map.values())

In [62]:
def print_results(results):
    keys = ['arango_len', 'clickhouse_len', 'duplicates', 'common_len', 'left_only_len', 'right_only_len', 'mismatch_len']
    for k in keys:
        print(k, ': ', result[k])

In [63]:
def get_default_get_all_query(collection):
    return f'''
        for r in {collection} 
            limit 50000
            return r
    '''

In [ ]:
# Alias routes
result = compare_collection('AliasRoutes', ['Id', 'AliasRouteId', 'SourceRouteId', 'Country'])

In [ ]:
# AmenitiesTransactions routes
result = compare_collection('AmenitiesTransactions', ['Id', 'RbUserID', 'RouteId', 'Email', 'MobileNo', 'OperatorId',
                                                     'ServiceId', 'OrderItemReferenceNo', 'Amenities',
                                                     'DojInLong', 'DojEpoch', 'DepartueTime', 'ArrivalTime',
                                                     'UserName', 'Country'])

In [ ]:
print_results(result)

In [ ]:
# AmenitiesTransactions routes
result = compare_collection('AmenitiesFeedbackStatus', ['Id', 'UploadedOn', 'PublishedToAmenitiesQueueOn', 'PushNotificationTriggeredOn', 'ReceivedAmenitiesRating',
'ReceivedRatingOn', 'ReceivedRatingVia', 'PushNotificationStatus', 'DojEpoch', 'RouteID'])

In [ ]:
print_results(result)

In [ ]:
result['common']

In [ ]:
# Challenged reviews
print(get_ch_columns('ChallengedReviews'))

In [ ]:
result = compare_collection('ChallengedReviews', ['Id', 'Name', 'DOJInLocal', 'SourceLocation', 'DestinationLocation',
                                                  'Pnr', 'ServiceProviderID', 'RouteID', 'SourceID', 'DestinationID',
                                                  'SrcLocationID', 'DstLocationID', 'Value', 'RatingSubmittedOn',
                                                  'ModeratedReview', 'Tags', 'Sentiment', 'Country', 'ReviewComment',
                                                  'SubmittedEpoch', 'OpReply', 'Status', 'UpdatedEpoch'])

In [ ]:
print_results(result)

In [ ]:
result['common']

In [ ]:
# OperatorReply
print(get_ch_columns('OperatorReply'))

In [ ]:
result = compare_collection('OperatorReply',
                           '''
                               for r in OperatorReply 
                                filter r.DojEpoch >=  1602129385000
                                limit 50000
                                return r
                           ''',
                           ['Id', 'OperatorId', 'OperatorName', 'Status', 'ServiceID', 'OrderItemReferenceNo', 
                            'Reply', 'ReplyDate', 'ModeratedReply', 'ModeratedDate', 
                            'ModeratedDateInLong', 'ModeratedBy', 'DojEpoch', 'Country'])

In [ ]:
print_results(result)

In [ ]:
# Note:
# ReplyDateInLong is not captured in Arango

In [ ]:
# OpRatingRecency
print(get_ch_columns('OpRatingRecency'))

In [ ]:
result = compare_collection('OpRatingRecency', get_default_get_all_query('OpRatingRecency'),
                           ['Id', 'AverageRating', 'TotalRatings', 'IsUpdated'])

In [ ]:
print_results(result)

In [ ]:
result['common']

In [ ]:
# RatingRecency
print(get_ch_columns('RatingRecency'))

In [ ]:
result = compare_collection('RatingRecency', get_default_get_all_query('RatingRecency'),
                           ['RouteId', 'TotalRatings', 'AverageRating', 'Country', 'OneRatingPerc', 
                            'TwoRatingPerc', 'ThreeRatingPerc', 'FourRatingPerc', 'FiveRatingPerc',
                            'OperatorId'])

print_results(result)

In [ ]:
# RatingRecency_LATAM
print(get_ch_columns('RatingRecency_LATAM'))

In [ ]:
result = compare_collection('RatingRecency_LATAM', get_default_get_all_query('RatingRecency_LATAM'),
                           ['RouteId', 'TotalRatings', 'AverageRating', 'Country', 'OneRatingPerc', 
                            'TwoRatingPerc', 'ThreeRatingPerc', 'FourRatingPerc', 'FiveRatingPerc',
                            'OperatorId'])

print_results(result)

In [ ]:
# RatingRecency_SEA
print(get_ch_columns('RatingRecency_SEA'))

In [ ]:
result = compare_collection('RatingRecency_SEA', get_default_get_all_query('RatingRecency_SEA'),
                           ['RouteId', 'TotalRatings', 'AverageRating', 'Country', 'OneRatingPerc', 
                            'TwoRatingPerc', 'ThreeRatingPerc', 'FourRatingPerc', 'FiveRatingPerc',
                            'OperatorId'])

print_results(result)

In [ ]:
# RatingRecency_RDC
print(get_ch_columns('RatingRecency_RDC'))

In [ ]:
result = compare_collection('RatingRecency_RDC', get_default_get_all_query('RatingRecency_RDC'),
                           ['Id', 'RouteId', 'TotalRatings', 'AverageRating', 'Country', 'OneRatingPerc', 
                            'TwoRatingPerc', 'ThreeRatingPerc', 'FourRatingPerc', 'FiveRatingPerc',
                            'OperatorId'])

print_results(result)

In [ ]:
# RestStopDetails
print(get_ch_columns('RestStopDetails'))

In [ ]:
result = compare_collection('RestStopDetails', get_default_get_all_query('RestStopDetails'),
                            ['RestStopId', 'Latitude', 'Longitude', 'ArrivalTime', 'LocationName', 'Duration', 
                            'RatingCount', 'UploadedOn'])

print_results(result)

In [ ]:
# RestStopDetails_Response
print(get_ch_columns('RestStopDetails_Response'))

In [ ]:
result = compare_collection('RestStopDetails_Response', get_default_get_all_query('RestStopDetails_Response'),
                            ['Id', 'OrderItemUUID', 'RestStopID', 'RouteID', 'DojEpoch', 'ServiceProviderID', 
                             'SubmittedOn'])

print_results(result)

In [ ]:
# UserRatings
print(get_ch_columns('UserRatings'))

In [ ]:
result = compare_collection('UserRatings',
                            '''
                                for r in UserRatings
                                    filter r.DojEpoch >= 1638993971
                                    limit 50000
                                    return r
                            ''',
                            ['Id', 'Value', 'RatingSubmittedOn', 'RatingDateOfSubmitInLong', 'RatingEpoch', 
                             'IsNeonUpdated', 'RbUserID', 'RbUserIDHash', 'MobileNo', 'Email', 
                             'TotalCustomerPayable', 'SourceLocation', 'ServiceProviderName', 'RouteID', 
                             'OrderItemReferenceNo', 'DestinationLocation', 'DateOfService', 'DateOfPurchase', 
                             'DepartueTime', 'ArrivalTime', 'Country', 'SourceID', 'DestinationID', 'ServiceProviderID', 'NoOfSeats', 'Status', 'OrgUnit', 
                             'DroppingPoIntID', 'BoardingPoIntID', 'DroppingPoInt', 'BoardingPoInt', 
                             'Name', 'Gender', 'Pnr', 'ServiceID', 'SalesChannel', 'OrderItemID', 'OrderItemUUID',
                             'OrderID', 'OrderUUID', 'IsOptInForWhatsapp', 'SelectedLanguage', 'JourneyType',
                             'IsGPSEnabled', 'DOJInLocal', 'BusType', 'SeatName', 'DojInLong', 'DojEpoch', 'Age', 
                             'BusTypeId', 'SrcLocationId', 'DstLocationId', 'Amenities'])

print_results(result)


In [ ]:
# UserReviews
print(get_ch_columns('UserReview'))

In [ ]:
result = compare_collection('UserReview',
                            '''
                                for r in UserReview
                                    filter r.DojEpoch >= 1638993971
                                    limit 50000
                                    return r
                            ''',
                            ['Id', 'Review', 'ReviewSubmittedOn', 'ReviewDateOfSubmitInLong', 
                             'DateOfModifiedInLong', 'ModeratedReview', 'ModeratedReviewSubmittedOn', 
                             'ModeratedDateInLong', 'ModeratedBy', 'Status', 'DojEpoch', 'RatingEpoch', 
                             'Country', 'HelpfulCount', 'ViewCount', 'PredictedTags', 'SelectedTags'])

print_results(result)


In [ ]:
# UserReviewTags
# Can't validate now need special case handling for ReviewTags
print(get_ch_columns('UserReviewTags'))

In [ ]:
result = compare_collection('UserReviewTags',
                            '''
                                for r in UserReviewTags
                                    filter r.DojEpoch >= 1638993971
                                    limit 50000
                                    return r
                            ''',
                            ['Id', 'SubmittedOn', 
                             'SubmitInLong', 'DojEpoch', 'Country'])

print_results(result)


In [89]:
# User
print(get_ch_columns('User'))

({'_key': 'UserId', 'UserIdHash': 'UserIdHash', 'UserName': 'UserName', 'PhoneCode': 'PhoneCode', 'Mobile': 'Mobile', 'Email': 'Email', 'ModifiedOn': 'ModifiedOn', '_sign': '_sign', '_ver': '_ver'}, ['UserId', 'UserIdHash', 'UserName', 'PhoneCode', 'Mobile', 'Email', 'ModifiedOn', '_sign', '_ver'])


In [90]:
result = compare_collection('User', get_default_get_all_query('User'), ['UserId'])

print_results(result)

read 2
read 3
read 4
read 5
['UserId', 'UserIdHash', 'UserName', 'PhoneCode', 'Mobile', 'Email', 'ModifiedOn', '_sign', '_ver']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   UserId          50000 non-null  object
 1   _id             50000 non-null  object
 2   _rev            50000 non-null  object
 3   UserId          50000 non-null  object
 4   IsUnSubscribed  19 non-null     object
 5   UpdatedOn       19 non-null     object
dtypes: object(6)
memory usage: 2.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   UserId      50000 non-null  int64 
 1   UserIdHash  50000 non-null  object
 2   UserName    50000 non-null  object
 3   PhoneCode   50000 non-null  object
 4   Mobile      50

In [66]:
result['left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 49999 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   UserId  1 non-null      int64   
 1   _merge  1 non-null      category
dtypes: category(1), int64(1)
memory usage: 121.0 bytes
